In [1]:
import json
import numpy as np
from pandas.io.json import json_normalize

import os
import glob as gb
import tensorflow as tf

from sklearn.model_selection import train_test_split
import keras 
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM , Flatten , CuDNNLSTM , GRU, Bidirectional, Conv1D, MaxPooling1D
from keras.layers import Dropout
import keras

from keras import callbacks
from keras.callbacks import  CSVLogger


# Model Evaluations
from sklearn import metrics
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold

In [2]:
def load_file(filepath,i):
    with open(filepath) as f:
        data = json.load(f)
    imu = data['imu']['data']
    emg = data['emg']['data']
    emg = np.array(emg)
    imu = np.array(imu)
    imu_gyr = np.array([(e['gyroscope']) for e in imu])
    imu_acc = np.array([(e['acceleration']) for e in imu])
    imu_orn = np.array([(e['orientation']) for e in imu])
    #timestamp = [i]
    #timestamp = np.repeat(timestamp, 400, axis=None)
    #timestamp= timestamp.reshape(400,1)
    dataset = tf.concat([emg, imu_gyr, imu_acc, imu_orn], axis=1, name='concat')
    dataset = np.array(dataset)
    return dataset

In [3]:
data_path = '../input/emgimu/An-EMG-and-IMU-Dataset-for-the-Italian-Sign-Language-Alphabet-master/Dataset/'

In [4]:
# For Files Data
X = []
y = []
i=0
for folder in  os.listdir(data_path) : 
    j=0
    files = gb.glob(pathname= str( data_path  + folder + '/*.json'))
    for file in files: 
        data = load_file(file,j)
        X.append(data)
        y.append(i)
        j+=1
    i+=1

print(f'we have {len(X)} items in X ')

2022-10-26 21:21:29.975328: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 21:21:29.976270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 21:21:30.363479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 21:21:30.364313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 21:21:30.365067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

we have 780 items in X 


In [5]:
X = np.array(X)
y = np.array(y)

In [6]:
print('X shape is : ' , X.shape)
print('y shape is : ' , y.shape)

X shape is :  (780, 400, 18)
y shape is :  (780,)


In [7]:
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 
           'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 
           'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}

In [8]:
def getcode(n) : 
    for x , y in mapping.items() : 
        if n == y : 
            return x

In [9]:
new_X = []
new_y = []
for j in range(780):
    for i in range(400):
        new_X.append(X[j][i])
        new_y.append(y[j])

In [10]:
print(len(new_X))
print(len(new_y))

312000
312000


In [11]:
new_X = np.array(new_X)
new_y = np.array(new_y)

In [12]:
from sklearn.utils import shuffle
new_X, new_y = shuffle(new_X, new_y, random_state=20)

In [13]:
print('new_X shape is : ' , new_X.shape)
print('new_y shape is : ' , new_y.shape)

new_X shape is :  (312000, 18)
new_y shape is :  (312000,)


# Building Model with samples

In [14]:
new_X = new_X.reshape(len(new_X), new_X.shape[1],1)
new_y = to_categorical(new_y)

In [15]:
kfold = KFold(n_splits=10, shuffle=True)

In [23]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
for train, test in kfold.split(new_X, new_y):

  # Model architecture
    model = Sequential()
    model.add(Conv1D(128, 3, activation='relu', input_shape=(18, 1))),
    model.add(MaxPooling1D(3)),
    model.add(Conv1D(64, 3, activation='relu')),
    model.add(MaxPooling1D(3)),
    model.add(Bidirectional(CuDNNLSTM(units=265, return_sequences=True)))
    model.add(Dropout(rate=0.3))
    model.add(Bidirectional(CuDNNLSTM(units=128, return_sequences=True)))

    model.add(Flatten())
    model.add(Dense(26,activation='softmax'))


    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#     es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
#     checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="./checkpoint-{epoch:02d}.hdf5",
#                                                       verbose=1, save_best_only=True,
#                                                       monitor='val_acc',mode='max')
#     csv_logger = CSVLogger('training_set_iranalysis3.csv',separator=',', append=False)




    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    
    history = model.fit(new_X[train], new_y[train], 
                        batch_size=128, epochs=32)

    
    scores = model.evaluate(new_X[test], new_y[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; \
    {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/32
2194/2194 [==============================] - 59s 8ms/step - loss: 2.5598 - accuracy: 0.2133
Epoch 2/32
2194/2194 [==============================] - 17s 8ms/step - loss: 2.3726 - accuracy: 0.2622
Epoch 3/32
2194/2194 [==============================] - 17s 8ms/step - loss: 2.3125 - accuracy: 0.2790
Epoch 4/32
2194/2194 [==============================] - 17s 8ms/step - loss: 2.2777 - accuracy: 0.2894
Epoch 5/32
2194/2194 [==============================] - 17s 8ms/step - loss: 2.2531 - accuracy: 0.2965
Epoch 6/32
2194/2194 [==============================] - 16s 7ms/step - loss: 2.2352 - accuracy: 0.3011
Epoch 7/32
2194/2194 [==============================] - 17s 8ms/step - loss: 2.2198 - accuracy: 0.3057
Epoch 8/32
2194/2194 [==============================] - 17s 8ms/step - loss: 2.2060 - accuracy: 0.3097
Epoch 9/32
2194/2194 [==============================] - 17s 8ms/step - loss: 2.

In [24]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 2.1793060302734375 - Accuracy: 32.40705132484436%
------------------------------------------------------------------------
> Fold 2 - Loss: 2.1859073638916016 - Accuracy: 31.987178325653076%
------------------------------------------------------------------------
> Fold 3 - Loss: 2.1889734268188477 - Accuracy: 31.7724347114563%
------------------------------------------------------------------------
> Fold 4 - Loss: 2.206974744796753 - Accuracy: 31.39423131942749%
------------------------------------------------------------------------
> Fold 5 - Loss: 2.1707005500793457 - Accuracy: 32.83974230289459%
------------------------------------------------------------------------
> Fold 6 - Loss: 2.204838275909424 - Accuracy: 31.74038529396057%
------------------------------------------------------------------------
>